In [ ]:
1

In [ ]:
import conservatoire

In [ ]:
import numpy as np
import pandas as pd

pd.options.display.float_format = lambda v: "{:_.2f}".format(v).replace("_", " ")

In [ ]:
from openfisca_france import CountryTaxBenefitSystem

In [ ]:
base = CountryTaxBenefitSystem()
base.load_extension("openfisca_france_local")

In [ ]:
import sys

sys.path.append("../technique")
from utils import *

In [ ]:
import conservatoire

In [ ]:
res = conservatoire.get_results(base, 2)

In [ ]:
res[0]

In [ ]:
from utils import StrasbourgSurveyScenario

In [ ]:
conservatoire

In [ ]:
count = len(df)
sample_count = 1
individu_df = pd.DataFrame(
    {
        "famille_id": list(range(count * sample_count)),
    }
)
sample_ids = np.repeat(list(range(sample_count)), count)
indiv_ids = np.tile(list(range(count)), sample_count)
# sample_qfrule = np.tile(product_df.qfrule, sample_count)

famille_df = pd.DataFrame(
    {
        "sample_id": sample_ids,
        "strasbourg_conservatoire_base_ressources": np.tile(
            df["ressources"], sample_count
        ),
        "strasbourg_conservatoire_nombre_cycles": np.tile(df["Cycle.1"], sample_count),
        "agent_ems": np.tile(df.agent, sample_count),
        "habitant_ems": np.tile(df["habitant EMS"], sample_count),
    }
)

menage_df = pd.DataFrame({})
foyerfiscaux_df = pd.DataFrame({})

individu_df["famille_role_index"] = 0
individu_df["foyer_fiscal_id"] = individu_df.famille_id
individu_df["foyer_fiscal_role_index"] = 0
individu_df["menage_id"] = individu_df.famille_id
individu_df["menage_role_index"] = 0

data = dict(
    input_data_frame_by_entity=dict(
        individu=individu_df,
        famille=famille_df,
        menage=menage_df,
        foyer_fiscal=foyerfiscaux_df,
    )
)

In [ ]:
df = conservatoire.get_df()
data = conservatoire.build_data(df)

In [ ]:
scenario = StrasbourgSurveyScenario(base, data)

In [ ]:
names = [
    n
    for n in base.variables
    if n.startswith("strasbourg_conservatoire")
    and not n.endswith("base_ressources")
    and "nombre" not in n
]

In [ ]:
for n in names:
    v = scenario.simulation.calculate(n, base_period)
    df[n] = v

In [ ]:
res = (
    df[["index", "MontantFactureSurEleve", *names]]
    .set_index(["index", "MontantFactureSurEleve"])
    .stack()
    .reset_index()
    .rename(columns={0: "Montant calculé", "level_2": "variable"})
)
res

In [ ]:
res["match"] = (res["Montant calculé"] - res["MontantFactureSurEleve"]).abs() < 0.1

In [ ]:
res[res.match]  # .to_excel('crr_correspondances.xlsx')

In [ ]:
res[res.match].groupby(
    "index"
).variable.first()  # .to_excel('crr_correspondances_test.xlsx')

In [ ]:
pd.DataFrame({"match": res.groupby("index").match.sum()}).reset_index()

In [ ]:
row_data = pd.DataFrame({"match": res.groupby("index").match.sum()}).reset_index()

In [ ]:
row_data.groupby("match").count()

In [ ]:
rdf = df.merge(row_data, on="index")

In [ ]:
assert len(rdf[rdf.match == 0]) == 1